In [ ]:
# Listado de bibliotecas necesarias:

import pandas as pd
import requests

# Si no deja importar alguna de las librerías, probar a ejecutar el comando pertinente en la terminal:
#----- pip install pandas
#----- pip install requests

In [15]:
class Extraccion_Limpieza_MovieDatabase:
    
     """
    Clase para extraer datos de la base de datos de películas de MovieDatabase, realizar limpieza y guardar en formato CSV.

    Atributos:
        anio_inicial (int): Año inicial para la extracción de datos.
        anio_final (int): Año final para la extracción de datos.
        genero (str): Género de las películas a extraer.
        tipo (str): Tipo de título a extraer (ej. movie, short, etc.).
        key (str): Clave de la API para acceder a los datos.

    Métodos:
        extraer_datos: Extrae datos de la API de películas y los almacena en un diccionario.
        limpiar_datos: Limpia los datos extraídos y los prepara para su almacenamiento.
        guardar_csv: Guarda los datos limpiados en un archivo CSV.
    """
    
    def __init__ (self, anio_inicial, anio_final, genero , tipo , key):
        
        self.anio_inicial = anio_inicial
        self.anio_final = anio_final
        self.genero = genero
        self.tipo = tipo
        self.key = key

    def extraer_datos(self):
        
        """
        Extrae datos de la API de películas y los almacena en un diccionario.

        Returns:
            dict: Diccionario con los datos extraídos.
        """
        
        self.pelis = {}
        anio_distinto = False

        print(f"--- Extrayendo datos ---")
        print(" ")

        for anio in range(self.anio_inicial, self.anio_final+1): 

            print(f"Iterando por el año {anio}:")
            
            for pag in range(1, 51): # Itera por un rango determinado de páginas, ya que por cada página sólo se puede obtener un máximo de 50 entradas
                if anio_distinto: # Condición para que detenga el bucle si el año de lanzamiento es distinto al solicitado (variable = True)
                    break

                url = "https://moviesdatabase.p.rapidapi.com/titles"
                querystring = {"genre": self.genero, "startYear": anio, "endYear": anio, "page": pag, 'titleType' : self.tipo} # Parámetros de la URL (versión 1, si no funciona o devuelve menos resultados de lo esperado >>> reemplazar por versión 2)
                #versión 2: querystring = {"genre": self.genero, "year": anio, "page": pag, 'sort': 'year.incr' , 'titleType' : self.tipo}
                headers = {"X-RapidAPI-Key": self.key, "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}
                response = requests.get(url, headers=headers, params=querystring)
                response = response.json()

                try: # Intenta acceder al diccionario
                    response['results']

                except: # Si no puede acceder significa que la url ha devuelto otra cosa y que, probablemente, se haya llegado al máximo de solicituds permitidas por hora
                    print(f"Se ha llegado al límite de solicitudes. Deteniendo la iteración")
                    break # Si se ha llegado al límite de solicitudes, detiene el bucle para que este no dé error

                # Revisa si la página que ha devuelto la API coincide con la página solicitada (se ha detectado que en el caso de no encontrar una página, la API devuelve la 1 por defecto y eso provocaba repeticiones de datos). 
                if response.get('page') != str(pag):
                    print(f"La página {pag} no existe. Deteniendo la iteración.")
                    break # Si los datos no coinciden, detiene el bucle

                # Revisa si no encuentra más entradas y en caso afirmativo detiene el bucle
                if len(response['results']) == 0:
                    print(f"Ya no hay más resultados. Deteniendo la iteración")
                    break
                
                # Itera por cada película/serie/elemento dentro de la lista de resultados
                for pelicula in response['results']:
                    release_year = pelicula.get('releaseYear') # Asigna el valor de releaseYear a una variable

                    # Revisa si el valor de releaseYear no es un None (es decir, existe) y si es distinto al año solicitado. En caso de que así sea, detiene el bucle
                    if release_year is not None and pelicula['releaseYear']['year'] != anio:
                        print(f"Se ha detectado un año diferente a {anio}. Deteniendo la iteración")
                        anio_distinto = True  # Asigna el valor True a la variable anio_distinto para que sirva como parada del primer bucle for
                        break

                # Añade los resultados encontrados si ninguna de las anteriores condiciones para detener el bucle se ha cumplido y continúa con la búsqueda
                self.pelis[(anio, pag)] = response['results']

            print(" ")

        print(f"--- Extracción de datos finalizada ---")
        return self.pelis
    
    def limpiar_datos(self):
        
        """
        Limpia los datos extraídos y los prepara para su almacenamiento.

        Returns:
            list: Lista de tuplas con los datos limpiados.
        """
        
        self.peliculas = []
        
        # Itera por cada clave del diccionario, es decir: Va entrando en 2000, 1 > 2000 , 2 ... 
        for anio_pagina in self.pelis:
            
            # Itera por cada uno de los valores que hay por página y año
            for elemento in self.pelis[anio_pagina]:
                    
                if elemento['titleType']['id'] == 'movie':
                    
                    tipo = 'Pelicula'
                    
                elif elemento['titleType']['id'] == 'short':
                    
                    tipo = 'Cortometraje'
            
                nombre = elemento['titleText']['text'] # Agrega el título de la película o el cortometraje
                anio_estreno = elemento['releaseYear']['year'] # Agrega el año de estreno
                id_pelicula = elemento['id'] # Agrega el valor de la ID
                release_date = elemento.get('releaseDate') # Asigna el valor de releaseDate a una variable 
                
                if release_date is not None: # Revisa que el valor de release_date no sea un None
                    mes_estreno = elemento['releaseDate']['month']  # Si no es un None, agrega el mes de estreno
                    
                else:
                    mes_estreno = "-" # Si el valor de release_date es un None, agrega "-" ya que no tenemos el dato del mes
                
                pelicula_corto = (tipo , nombre , anio_estreno , mes_estreno , id_pelicula) # Crea una tupla con todos los elementos de la película o el cortometraje
                self.peliculas.append(pelicula_corto) # Añade la tupla a la lista de tuplas
                
        print(f' ')    
        print(f'La limpieza de datos se ha completado con éxito')
        return self.peliculas
    
    def guardar_csv (self):
        
        """
        Guarda los datos limpiados en un archivo CSV.
        """
    
        df_dict_json = pd.DataFrame(self.peliculas)
        
        nombre = input("Indica el nombre del archivo. Recuerda poner .csv al final")
        
        df_dict_json.to_csv(nombre)
        
        print(f' ')    
        print(f'El archivo se ha guardado como ruta/nombre: {nombre}')

In [14]:
#! EXTRACCIÓN DE PELÍCULAS - DRAMA

# Llamamos a la clase y le damos los atributos necesarios: año inicial, año final, género, tipo (movie o short) y la key personal de MovieDatabase)
Extraccion_pelis_drama = Extraccion_Limpieza_MovieDatabase(2020 , 2023 , 'Drama' , 'movie' , 'b38cd0a830msh149cd093d780ffcp1fa141jsnc85ccd4fb846')

# Extraemos los datos llamando a la función de clase y asignamos los resultados a una variable
datos_pelis_drama = Extraccion_pelis_drama.extraer_datos()

# Limpiamos los datos extraídos llamando a la función de clase y asignamos los resultados a una variable
pelis_drama = Extraccion_pelis_drama.limpiar_datos()

# Guardamos el resultado en un csv
Extraccion_pelis_drama.guardar_csv() # Ejemplo de input usando ruta del archivo: Drama/pelis_drama_2020_2023.csv

--- Extrayendo datos ---
 
Iterando por el año 2020:
 
Iterando por el año 2021:
 
Iterando por el año 2022:
 
Iterando por el año 2023:
 
--- Extracción de datos finalizada ---
 
La limpieza de datos se ha completado con éxito
 
El archivo se ha guardado como ruta/nombre: Drama/prueba.csv


In [ ]:
#! EXTRACCIÓN DE CORTOS - DRAMA

# Llamamos a la clase y le damos los atributos necesarios: año inicial, año final, género, tipo (movie o short) y la key personal de MovieDatabase)
Extraccion_cortos_drama = Extraccion_Limpieza_MovieDatabase(2020 , 2023 , 'Drama' , 'short' , 'b38cd0a830msh149cd093d780ffcp1fa141jsnc85ccd4fb846')

# Extraemos los datos llamando a la función de clase y asignamos los resultados a una variable
datos_cortos_drama = Extraccion_cortos_drama.extraer_datos()

# Limpiamos los datos extraídos llamando a la función de clase y asignamos los resultados a una variable
cortos_drama = Extraccion_cortos_drama.limpiar_datos()

# Guardamos el resultado en un csv
Extraccion_cortos_drama.guardar_csv() # Ejemplo de input usando ruta del archivo: Drama/cortos_drama_2020_2023.csv

In [ ]:
#! EXTRACCIÓN DE PELÍCULAS - COMEDIA

# Llamamos a la clase y le damos los atributos necesarios: año inicial, año final, género, tipo (movie o short) y la key personal de MovieDatabase)
Extraccion_pelis_comedia = Extraccion_Limpieza_MovieDatabase(2020 , 2023 , 'Comedy' , 'movie' , 'b38cd0a830msh149cd093d780ffcp1fa141jsnc85ccd4fb846')

# Extraemos los datos llamando a la función de clase y asignamos los resultados a una variable
datos_pelis_comedia = Extraccion_pelis_comedia.extraer_datos()

# Limpiamos los datos extraídos llamando a la función de clase y asignamos los resultados a una variable
pelis_comedia = Extraccion_pelis_comedia.limpiar_datos()

# Guardamos el resultado en un csv
Extraccion_pelis_comedia.guardar_csv() # Ejemplo de input usando ruta del archivo: Comedia/pelis_comedia_2020_2023.csv

In [ ]:
#! EXTRACCIÓN DE CORTOS - COMEDIA

# Llamamos a la clase y le damos los atributos necesarios: año inicial, año final, género, tipo (movie o short) y la key personal de MovieDatabase)
Extraccion_cortos_comedia = Extraccion_Limpieza_MovieDatabase(2020 , 2023 , 'Comedy' , 'short' , 'b38cd0a830msh149cd093d780ffcp1fa141jsnc85ccd4fb846')

# Extraemos los datos llamando a la función de clase y asignamos los resultados a una variable
datos_cortos_comedia = Extraccion_cortos_comedia.extraer_datos()

# Limpiamos los datos extraídos llamando a la función de clase y asignamos los resultados a una variable
cortos_comedia = Extraccion_cortos_comedia.limpiar_datos()

# Guardamos el resultado en un csv
Extraccion_cortos_comedia.guardar_csv() # Ejemplo de input usando ruta del archivo: Comedia/cortos_comedia_2020_2023.csv

In [ ]:
#! EXTRACCIÓN DE PELÍCULAS - ACCIÓN

# Llamamos a la clase y le damos los atributos necesarios: año inicial, año final, género, tipo (movie o short) y la key personal de MovieDatabase)
Extraccion_pelis_accion = Extraccion_Limpieza_MovieDatabase(2020 , 2023 , 'Action' , 'movie' , 'b38cd0a830msh149cd093d780ffcp1fa141jsnc85ccd4fb846')

# Extraemos los datos llamando a la función de clase y asignamos los resultados a una variable
datos_pelis_accion = Extraccion_pelis_accion.extraer_datos()

# Limpiamos los datos extraídos llamando a la función de clase y asignamos los resultados a una variable
pelis_accion = Extraccion_pelis_accion.limpiar_datos()

# Guardamos el resultado en un csv
Extraccion_pelis_accion.guardar_csv() # Ejemplo de input usando ruta del archivo: Accion/pelis_accion_2020_2023.csv

In [ ]:
#! EXTRACCIÓN DE CORTOS - ACCIÓN

# Llamamos a la clase y le damos los atributos necesarios: año inicial, año final, género, tipo (movie o short) y la key personal de MovieDatabase)
Extraccion_cortos_accion = Extraccion_Limpieza_MovieDatabase(2020 , 2023 , 'Action' , 'short' , 'b38cd0a830msh149cd093d780ffcp1fa141jsnc85ccd4fb846')

# Extraemos los datos llamando a la función de clase y asignamos los resultados a una variable
datos_cortos_accion = Extraccion_cortos_accion.extraer_datos()

# Limpiamos los datos extraídos llamando a la función de clase y asignamos los resultados a una variable
cortos_accion = Extraccion_cortos_accion.limpiar_datos()

# Guardamos el resultado en un csv
Extraccion_cortos_accion.guardar_csv() # Ejemplo de input usando ruta del archivo: Accion/cortos_accion_2020_2023.csv